In [48]:
import cv2
import face_recognition
import matplotlib.pyplot as plt
import dlib
import numpy
import os

#顔全体とマスク着用時の各種割合を検証する
#DataSetディレクトリに個人の顔と目のデータが保存されている

num=0
flag = 0
acc = 0
loss = 0

#動画の読み込み　パスは適宜変更すること
#これが認証する際の顔画像となる
cap = cv2.VideoCapture('./kai/2.mp4')#顔が映っている動画（3~5s）

#読み込めていたらTrue
print(cap.isOpened())

#顔のカスケードファイル
#face_cascade_file = 'haarcascade_frontalface_default.xml'
face_cascade_file = 'haarcascade_frontalface.xml'
#両目のカスケードファイル
eyepair_cascade_file = 'build_share_OpenCV_haarcascades_haarcascade_mcs_eyepair_big.xml'
#口のカスケードファイル
mouth_cascade_file = 'haarcascade_mcs_mouth.xml'

#顔のカスケード分類器
face_cascade = cv2.CascadeClassifier(face_cascade_file)
#両目のカスケード分類器
eyes_cascade = cv2.CascadeClassifier(eyepair_cascade_file)
#口のカスケード分類器
mouth_cascade = cv2.CascadeClassifier(mouth_cascade_file)

#dlibの顔検出器
detector = dlib.get_frontal_face_detector()


#顔はFaceRecognition
#目はAKAZE


#ここからFaceRecognitionの準備
#保存されている人物の顔の画像を読み込む。
known_face_imgs = []
#データセットのあるパス 試験的に自分の顔画像を使う（約5枚）
FACE_IMG_DIR = os.path.abspath(os.path.dirname("__file__")) + '/DataSet/kai/face/'
EYE_IMG_DIR = os.path.abspath(os.path.dirname("__file__")) + '/DataSet/kai/eye/'

face_files = os.listdir(FACE_IMG_DIR)
eye_files = os.listdir(EYE_IMG_DIR)
for file in face_files:
    if file == '.DS_Store':
        continue

for path in face_files:
    img = face_recognition.load_image_file(FACE_IMG_DIR + path)
    known_face_imgs.append(img)
#ここまで

#AKAZEの準備
bf = cv2.BFMatcher(cv2.NORM_HAMMING)
# 特徴量記述
akaze_detector = cv2.AKAZE_create()
#ここまで

#c：認証回数
c = 0
i = 0

#この中にAKAZEの特徴量マッチングを行う処理を入れる
while True:
    ret, frame = cap.read()
    #90度反転
    frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
    if ret:
        c = c+1
        #全部は多いので試験的に10フレーム分処理を行う
        if c==11:
            break
            
        #dlibの顔検出器で顔を検出し、それをfacesに保存する。
        faces = detector(frame)
        
        #顔の検出
        #face = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=1,minSize=(200,200))
        face = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=10)
        #口の検出
        mouth = mouth_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=80)
        
        #顔が検出できるか
        if len(faces)< 0 and len(face)>0:
            #顔を検出
            print("顔が映っています")
            #print(face)
            #print(mouth)
            #jpgファイルにしないとエラーが出るため，一旦保存して変換
            tmp = cv2.imwrite("frame.jpg",frame)

            #顔認証（FaceRecognition）の処理
            # 認証する人物の顔の画像を読み込む。
            face_img_to_check = face_recognition.load_image_file("frame.jpg")
            
            # 顔の画像から顔の領域を検出する。
            known_face_locs = []
            for img in known_face_imgs:
                loc = face_recognition.face_locations(img, model="hog")
                assert len(loc) == 1, "画像から顔の検出に失敗したか、2人以上の顔が検出されました"
                known_face_locs.append(loc)

            face_loc_to_check = face_recognition.face_locations(face_img_to_check, model="hog")
            assert len(face_loc_to_check) == 1, "画像から顔の検出に失敗したか、2人以上の顔が検出されました"

            # 顔の領域から特徴量を抽出する。
            known_face_encodings = []
            for img, loc in zip(known_face_imgs, known_face_locs):
                (encoding,) = face_recognition.face_encodings(img, loc)
                known_face_encodings.append(encoding)

            (face_encoding_to_check,) = face_recognition.face_encodings(
                face_img_to_check, face_loc_to_check
            )
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding_to_check,tolerance = 0.4)
            print(matches)
            dists = face_recognition.face_distance(known_face_encodings, face_encoding_to_check)
            print(dists) 
            #print(known_face_locs)
#             print(face)
#             print(mouth)

        else:
            #顔が検出されなかったら
            #目の検出
            #eyes = eyes_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=1)
            #minNeighborsの値を高くすることで信頼性が増す
            eyes = eyes_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=1)
            #画像のサイズを正規化
            IMG_SIZE = (600, 150)
            #目が検出されたら
            if len(eyes) > 0:
                for x,y,w,h in eyes:
                    eyes_cut = frame[y:y+h, x:x+w]
                    eyes_cut = cv2.resize(eyes_cut,IMG_SIZE)
                #AKAZEの処理
                print("マスクを着用していますね？")
                #eyes_cut = cv2.Canny(eyes_cut, 100, 200)#Canny(エッジ検出)
                tmp = cv2.imwrite("frame.jpg",eyes_cut)
                target_img = cv2.imread("frame.jpg")
                target_img = cv2.cvtColor(target_img, cv2.COLOR_BGR2GRAY)#白黒
                #target_img = cv2.Canny(target_img, 0, 200)#Canny(エッジ検出)
                cv2.imwrite(os.path.join("kakuninn.jpg"),target_img)
                #画像サイズの正規化
                target_img = cv2.resize(target_img, IMG_SIZE)
                #入力画像の特徴量を抽出
                (target_kp, target_des) = akaze_detector.detectAndCompute(target_img, None)
                for file in eye_files:
                    if file == '.DS_Store':
                        continue

                    comparing_img_path = EYE_IMG_DIR + file
                    try:
                        
                        #データセットの目の画像の読み込み
                        comparing_img = cv2.imread(comparing_img_path)
                        comparing_img = cv2.cvtColor(comparing_img, cv2.COLOR_BGR2GRAY)#白黒
                        #comparing_img = cv2.Canny(comparing_img, 0, 200)#Canny(エッジ検出)
                        #comparing_img = cv2.resize(comparing_img,(1000,1000))
#                         eyes = eyes_cascade.detectMultiScale(comparing_img, scaleFactor=1.1, minNeighbors=1)
#                         for x,y,w,h in eyes:
#                             comparing_img = comparing_img[y:y+h, x:x+w]
                        #comparing_img =  cv2.cvtColor(comparing_img,  cv2.COLOR_RGB2BGR)
                        comparing_img = cv2.resize(comparing_img, IMG_SIZE)
                        tmp = cv2.imwrite(os.path.join("result", "me"+str(i)+".jpg"), comparing_img)
                        i = i+1
                        (comparing_kp, comparing_des) = akaze_detector.detectAndCompute(comparing_img, None)
                        if (not target_des is None) and (not comparing_des is None):
                            matches = bf.match(target_des, comparing_des)
                            dist = [m.distance for m in matches]
                            #print(dist)
                            ret = sum(dist) / len(dist)
                        else:
                            #入力画像の特徴量がNONE
                            #エラー回避
                            print("error")
                            ret = 10000
                    except cv2.error:
                        ret = 10000

                    #num += ret
                    if ret<100:
                        #print("認証成功")
                        flag = 1
                        
                    print(file, ret)
#                 count = 0
#                 #print(sum(os.path.isfile(os.path.join("test", name)) for name in os.listdir("test"))) テストフォルダ内のファイル数
#                 print("平均値：")
#                 print(num/sum(os.path.isfile(os.path.join("test", name)) for name in os.listdir("test")))
#                 num = 0
                
            else:
                #目も検出されなかったら
                print("顔が検出されませんでした")
            if flag ==1:
                print("認証成功")
                flag = 0
                acc = acc+1
            else:
                print("認証失敗")
                loss = loss+1
            print(acc)
            print(loss)
    else:
        break

True
マスクを着用していますね？
kai_eye_000.jpg 216.0
kai_eye_001.jpg 187.0
kai_eye_002.jpg 203.0
kai_eye_003.jpg 190.0
kai_eye_005.jpg 220.0
kai_eye_006.jpg 221.0
kai_eye_007.jpg 177.0
kai_eye_008.jpg 190.0
kai_eye_009.jpg 216.0
kai_eye_010.jpg 191.0
認証失敗
0
1
マスクを着用していますね？
kai_eye_000.jpg 100.0
kai_eye_001.jpg 98.0
kai_eye_002.jpg 103.0
kai_eye_003.jpg 92.0
kai_eye_005.jpg 100.0
kai_eye_006.jpg 112.0
kai_eye_007.jpg 110.0
kai_eye_008.jpg 112.0
kai_eye_009.jpg 107.0
kai_eye_010.jpg 95.0
認証成功
1
1
マスクを着用していますね？
kai_eye_000.jpg 119.0
kai_eye_001.jpg 115.0
kai_eye_002.jpg 118.0
kai_eye_003.jpg 109.0
kai_eye_005.jpg 115.0
kai_eye_006.jpg 127.0
kai_eye_007.jpg 125.0
kai_eye_008.jpg 128.0
kai_eye_009.jpg 122.0
kai_eye_010.jpg 124.0
認証失敗
1
2
マスクを着用していますね？
error
kai_eye_000.jpg 10000
error
kai_eye_001.jpg 10000
error
kai_eye_002.jpg 10000
error
kai_eye_003.jpg 10000
error
kai_eye_005.jpg 10000
error
kai_eye_006.jpg 10000
error
kai_eye_007.jpg 10000
error
kai_eye_008.jpg 10000
error
kai_eye_009.jpg 10000
err

In [49]:
import dlib

image = cv2.imread("frame.jpg")

#dlibのget_frontal_face_detectorのインスタンスを立てる。
detector = dlib.get_frontal_face_detector()

face =detector(image)

print("{} face".format(len(face)))

for f in face:
        print("left, top, Right, bottom : ", f.left(), f.top(), f.right(), f.bottom())
        cv2.rectangle(img, (f.left(), f.top()), (f.right(), f.bottom()), (0,0,255), 2)

photo = dlib.image_window()
photo.set_image(image)
photo.add_overlay(face)
cv2.imwrite("frame.jpg", img)

1 face
left, top, Right, bottom :  403 760 938 1295


True

In [68]:
import cv2 

img = cv2.imread("frame.jpg")

img = cv2.Canny(img, 50, 50)#Canny(エッジ検出)

cv2.imwrite("canny.jpg",img)

True

In [63]:
import dlib
#minNeighborsは36がよさそう

#image = cv2.imread("./frame/test_0.jpg")
image = cv2.imread("./t/0.jpg")

face_cascade_file = 'haarcascade_frontalface.xml'

#image = cv2.resize(image,(400,600))
#顔の検出
face = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=36)

# 検出結果のリストから、位置・及び大きさを取得
for (x, y, w, h) in face:
    #矩形表示
    image = cv2.rectangle(image, (x, y), (x + w, y + h), (0,255,0), 2)
    #cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)

cv2.imwrite("k.jpg", image)

True